# BQP in PSPACE


:::{admonition} Extra Credit
:class: tip
Everything on this page is extra credit worth **20 points**.
:::

In this section, we are going to build Simulator 4. Our previous simulators were exponential in both time and space (RAM).  This simulator is going to be special because it is going to be (at least) exponential in time but it's going to require only a polynomial amount of space.   In fact, it uses so little space that you'd be able to run very large calculations on your laptop as long as you'd be willing to wait a really long time for your result.

The class which can be solved with polynomial amount of space but arbitrary computation (i.e. your laptop with the same RAM but really really fast clock speed) is class PSPACE.

By building simulator 4 (and showing that it works in polynomial space) you will have demonstrated that BQP in PSPACE. 

We need to start by taking some care with what we mean by running a quantum computation.  Previously, we've allowed our simulator to output the state vector (something a real quantum computer can't do but we could).  But the state vector is already exponentially large so we can't even print it out in polynomial time.   Instead, we will require this simulator to only work when MEASURE is applied. 

We will worry about figuring out how to implement the measure later.  Let's start by figuring out how to evaluate one amplitude - i.e. what's the amplitude of $|101\rangle$ - using only polynomial space. 

## An amplitude in Polynomial Space

First notice that in Simulation II, the thing that is cost you RAM is that you are doubling your list length every time you do a Hadamard.  Nothing but the Hadamard is causing trouble.


To build this simulator, we will make the following changes.
Get rid of the AddDuplicates function in simulator II.  Notice that this uses more RAM so seems to be going backwards. 


:::{admonition} Grading
:class: caution

Go ahead and implement this code.  Verify that you still get the same answer that you previously did (i.e. the add duplicates function was an efficiency improvement but not important). 

:::


Currently, we are using a lot of RAM.  Now we are going to make a change to our simulation.  Our simulation (which we will use as a subroutine for a bigger simulation) is going to take (1) a quantum circuit (which has $k$ Hadamards in it and (2) a binary number of length $k$.  For example, if we had four Hadamards, our simulator might take the number $1011$.  We will interpret this as only taking the "1" path of the first Hadamard, the "0" path of the second Hadamard, the "1" path of the third Hadamard and the "1" path of the fourth Hadamard.  


Recall that your Hadamard function `Hadamard(input,wire)` takes $|i\rangle \rightarrow \frac{1}{\sqrt{2}}\left(|0\rangle + (-1)^i |1\rangle  \right)$


:::{admonition} Grading
:class: caution

Make a new Hadamard function `Hadamard(input,wire,s)` which takes $|i\rangle \rightarrow \frac{1}{\sqrt{2}}\left(|0\rangle + (-1)^i |1\rangle  \right)$ and the bit `s` and returns the correct result. 

:::

Notice that when you run this new simulation it gives the wrong answer, but it doesn't take very much RAM at all.  The only place you were previously consuming RAM was that your Hadamard gate was doubling the list length each time.  Now it's not. 


---



### Putting it together

Currently we are using less RAM but getting the wrong answer.  Now, we want to go ahead and change things so we can get the correct answer and still not use much RAM.  Let's say we are trying to figure out the amplitude for $|101\rangle$.  Let's run our new simulation using all possible binary numbers (for the choices for our Hadamard).  Each one of those simulations is going to give us a value for the amplitude.  We should sum them all.  This is going to require an exponential number, $2^k$,  runs of our simulation but each one takes very little RAM (we will count carefully in a minute).  

We now have an algorithm which gives us the amplitude for a particular binary number.

:::{admonition} Grading
:class: caution

Go ahead and implement in your code.  Run a circuit of four Hadamards and compute the amplitude for $|101\rangle$. Check that you get the same result for that amplitude using Simulator II.  

:::

## Measuring in Polynomial Time

We now are able to get a particular amplitude in polynomial time.  As the next step, we'd like to be able to implement the MEASURE function. 

Before you were probably applying MEASURE by using `np.choice` with the state vector.  We obviously can't do that here since we can't afford to write out the state vector.  We are going to need to think more carefully about how to perform a MEASURE without having the whole vector.  

Essentially, we are going to invert things.  Let's first choose a random number $r \in (0,1)$.  Then we are just going to enumerate over the different binary numbers $|i\rangle$ computing the amplitude (and then the probability) for each one.  We can then subtract off the probability from our random number until we find the amplitude that makes it zero (be careful here about off-by-one errors). 

:::{admonition} Grading
:class: caution

Write code to successfully perform measure where you see one amplitude at a time.  


```
# do something (probably including pick a random number)
for amp in stateVector:
    # you can do whatever you want here 
    # except for store a lot of these amplitudes
    # return something 

```
:::


---

<!-- Now we need to figure out how to get a specific amplitude.  Maybe the amplitude for the binary number $|01101\rangle$.  Once we've done this, then we can just loop over all the binary numbers and generate each of the amplitudes and use our MEASURE technique from above . --> 

<!-- 
To do this, we shoudl think about our Simulator II.  Remember we have three gates: H, CNOT, $P(\theta)$.  Two of these gates - CNOT and $P(\theta)$ - don't increase the amount of RAM That is being used at all.  On the other hand, the Hadamard gate doubles the amount of RAM that is being used.  Every binary number in your list, gets broken into two additional binary numbers. 

We are going to modify simulator 2 in various ways.

Suppose you have a list of length $l$ and apply the Hadamard to gate $i$.
After the application (before you combine duplicates), you are going to have a list of length $2l$ with $l$ binary numbers  

Typically after the application of a gate, you would apply a `AddDuplicates` function.  While this is important to decrease the total amount of RAM being used, we don't have to do this to get a correct answer.  This is because quantum mechanics is linear - i.e. if your list is `[(a,'001'), (b,'010'), (c,'001')]` or your list is 
`[(a+c,'001'),(b,'010')]` you will get the same answer at the end of the day.  *Convince yourself of this*.   



:::{admonition} Grading
:class: caution

Now, run your code on the following circuit with and without the `AddDuplicates` function.  Verify that it gives the same amplitude for every vector.

(circuit here)

:::

-->

<!-- 
This seems to be heading in the wrong direction.   We saw two Hadamards where we could have potentially added some duplicates and we didn't choose to leaving us using more RAM. 

Let's consider the list that's produced immediately after the Hadamard acts on site $i$. 
After the application of the Hadamard (before you combine duplicates), you then have $l$ binary numbers with a 0 on site $i$ and $l$ binary numbers with a 1 on site $i$.  Let's sort your list so that these two lists are separate. 


Currently our simulators take as input simply the circuit (and the input state).  Suppose we also take as input whether each Hadamard should leave either the "0" or "1" bit generate from each Hadamard.  For example, we could run our circuit above as `Simulator([1,0])`.  Then the first Hadamard should leave the "1" part of the list and the second Hadamard should be the "0" part of your list. We could do this by not using the `AddDuplicate` function and instead 
use the 
```
def RemoveBit(i, s)
```
function which removes all the binary numbers with the i'th bit being s (either 0 or 1).



Notice that this simulator is not going to increase the RAM.  After the `RemoveBit` each list is still the same length.  

But now we are definitely going to get the wrong answer for our amplitude of $|01101\rangle$.  But suppose we run it four times returning
```
Simulator4([0,0])['01101']+Simulator4([0,1])['01101']+Simulator4([1,0])['01101']+Simulator4([1,1])['01101']
```
This will give you the amplitude of $|01101\rangle$.  



:::{admonition} Grading
:class: caution

Write code that takes a circuit and gets the amplitude of $|01101\rangle$.  It's going to have to look at the number of Hadamards and some over all of them.  Verify that it gets the right answer for this amplitude using your simulator.

Show that you're not using much RAM by printing out the average length of your list. 

(circuit here)

:::
-->

<!--
:::{admonition} Grading
:class: caution

Put it all together and write a code that succesfully does the MEASURE.  The computational complexity is now exponential in the number of Hadamards which is painful.   

:::
-->

<!--
It almost looks like you are working in constant space - your list never gets bigger.  But you have to store which Hadamard bits you are going to use.  This bit-string is linear in the number of Hadamards and the number of Hadamards could be polynomial in the number of qubits.  

-->